In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import tensorflow as tf
import keras
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.regularizers import l2
# Evaluation (Metrics & DM-Test)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from itertools import islice
from math import lgamma, fabs, isnan, nan, exp, log, log1p, sqrt
from typing import Sequence, Callable, List, Tuple
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

#### **1. Data Preparation**

In [3]:
df = pd.read_csv("df_ff_factors_100325.csv")
df = df.drop(columns=["Unnamed: 0","crsp_portno"])
df = df.sort_values(by='date')
df

df_tech = pd.read_csv("df_ff_factors_techfunds.csv")
df_tech = df_tech.drop(columns=["Unnamed: 0","crsp_portno"])
df_tech = df_tech.sort_values(by='date')

In [4]:
df_tech

,crsp_fundno,date,mth_return,exp_ratio,turn_ratio,normalised_flow,gdp_to_debt_ratio,gdp_growth_rate,unm_rate,infl_rate,...,cma,rf,umd,excess_return,rolling_sharpe,mkt_return,rolling_alpha_3f,rolling_alpha_4f,rolling_alpha_5f,shortrun_momentum
43631,32553.0,1993-08-31,0.068935,0.0162,0.15,0.009529,64.101,3.5225,6.8,2.90,...,-0.0004,0.0025,0.0265,0.068910,3.248937,0.056070,0.068910,0.068910,0.068910,0.012866
43632,32553.0,1993-09-30,-0.004082,0.0162,0.15,0.014881,64.101,3.5225,6.7,2.90,...,-0.0016,0.0026,0.0337,-0.004108,2.216452,0.027009,-0.004108,-0.004108,-0.004108,-0.009113
43633,32553.0,1993-10-29,0.040984,0.0162,0.15,0.030641,64.669,3.5225,6.8,2.90,...,0.0060,0.0022,-0.0272,0.040962,3.058133,0.021609,0.040962,0.040962,0.040962,0.000383
13747,12051.0,1993-10-29,0.034712,0.0188,0.77,0.070030,64.669,3.5225,6.8,2.90,...,0.0060,0.0022,-0.0272,0.034690,5.744340,0.021609,0.034690,0.034690,0.034690,0.013103
13748,12051.0,1993-11-30,-0.075974,0.0188,0.77,0.000259,64.669,3.5225,6.6,2.90,...,-0.0094,0.0025,-0.0474,-0.075999,-0.536935,-0.010806,-0.075999,-0.075999,-0.075999,-0.026033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031,4610.0,2024-06-28,0.078565,0.0217,0.23,-0.650182,120.040,2.5427,4.1,3.35,...,-0.0102,0.0047,0.0508,0.078518,1.619481,0.055058,0.001428,0.002329,0.003201,-0.025985
1960,4330.0,2024-07-31,0.028831,0.0077,0.31,-0.022800,120.731,2.5427,4.2,3.35,...,-0.0102,0.0047,0.0508,0.028784,0.691406,0.055058,0.004765,0.005643,0.010978,-0.024241
2113,4333.0,2024-07-31,0.028826,0.0076,0.31,-0.402964,120.731,2.5427,4.2,3.35,...,-0.0102,0.0047,0.0508,0.028779,0.691738,0.055058,0.004733,0.005601,0.010953,-0.024228
1842,4327.0,2024-07-31,0.028616,0.0102,0.31,-0.032107,120.731,2.5427,4.2,3.35,...,-0.0102,0.0047,0.0508,0.028569,0.676394,0.055058,0.004506,0.005384,0.010697,-0.024451


The chunk below describes the various functions involved in the training of the models, which are described in greater depth below:

(1) Generation of lagged dataframe i.e. lagging the characteristics

(2) Generation of stepped dataframe i.e. lagged characteristics being together with the 1-month ahead forecast

(3) Demeaning function for fund-level characteristics, which is important to ensure accuracy in the overall forecasting of annualised alphas

In [5]:
# Generate Lagged Dataset
def create_lagged_dataset(dataset, lag, target_var, id):
    lagged_dataset = dataset.copy()
    columns_list = list(lagged_dataset.columns)
    data_join = {}
    for column in columns_list:
        if column == target_var:
            data_join[column] = lagged_dataset[column]
        for n in range(1,lag+1):
            data_join[F'{column}_L{n}'] = lagged_dataset.groupby(id)[column].shift(n)
    lagged_dataset = pd.concat(data_join.values(), axis=1, ignore_index = True)
    lagged_dataset.columns = data_join.keys()
    return lagged_dataset.dropna()

# Generate Stepped Dataset for Training
## Steps is the number of months ahead that we are forecasting, e.g. step=2 is 2 months ahead.
## Note step=1 results in no change to dataset, i.e. use generated lagged variables to forecast current. 
def create_stepped_dataset(dataset, step, target_var, id):
    
    shifted_dataset = dataset.copy()
    shifted_dataset['shifted_target'] = shifted_dataset.groupby(id)[target_var].shift(-step + 1)
    
    # Drop rows where the shifted target is NaN (these occur due to the shift operation)
    shifted_dataset = shifted_dataset.dropna(subset=['shifted_target'])
    
    # Separate the features (X) and the target (y)
    X = shifted_dataset.drop(columns=[target_var, 'shifted_target'])
    y = shifted_dataset[['shifted_target']]
    y = y.rename(columns={'shifted_target':target_var})
    return X, y

def demeaning_fund_char(dataset, id, characteristic):
    for col in characteristic:
        dataset[f'demeaned_{col}'] = dataset[col] - dataset.groupby(id)[col].transform('mean')
    return dataset

In [6]:
import pandas as pd

# Assuming 'dataset' is your DataFrame and 'normalized_flow' is the column with NaN values
df['normalised_flow'] = df['normalised_flow'].fillna(method='ffill')
char_to_demean = ["exp_ratio", "turn_ratio", "normalised_flow", "shortrun_momentum"]
df_demeaned = demeaning_fund_char(df, id="crsp_fundno", characteristic = char_to_demean)
df_demeaned = df_demeaned.drop(columns = char_to_demean)   # drop original columns
df.head()

df_tech['normalised_flow'] = df_tech['normalised_flow'].fillna(method='ffill')
char_to_demean = ["exp_ratio", "turn_ratio", "normalised_flow", "shortrun_momentum"]
df_tech_demeaned = demeaning_fund_char(df_tech, id="crsp_fundno", characteristic = char_to_demean)
df_tech_demeaned = df_tech_demeaned.drop(columns = char_to_demean)   # drop original columns

df

C:\Users\wjlwi\AppData\Local\Temp\ipykernel_13784\1513914368.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['normalised_flow'] = df['normalised_flow'].fillna(method='ffill')
C:\Users\wjlwi\AppData\Local\Temp\ipykernel_13784\1513914368.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_tech['normalised_flow'] = df_tech['normalised_flow'].fillna(method='ffill')


,crsp_fundno,date,mth_return,exp_ratio,turn_ratio,normalised_flow,mtna,gdp_to_debt_ratio,gdp_growth_rate,unm_rate,...,rf,umd,mkt_return,excess_return,rolling_alpha_5f,shortrun_momentum,demeaned_exp_ratio,demeaned_turn_ratio,demeaned_normalised_flow,demeaned_shortrun_momentum
52070,21716.0,1993-05-28,-0.005886,0.0065,1.01,0.049932,124.190,63.915,3.5225,7.1,...,0.0022,0.0022,-0.037196,-0.008086,-0.008086,0.031310,-0.000387,0.433153,0.244742,0.037077
52071,21716.0,1993-06-30,0.023484,0.0065,1.01,0.036697,131.211,63.915,3.5225,7.0,...,0.0025,0.0455,-0.037196,0.020984,0.020984,0.045995,-0.000387,0.433153,0.231507,0.051762
13848,8441.0,1993-07-30,0.012705,0.0097,1.21,0.000000,511.692,64.101,3.5225,6.9,...,0.0024,0.0316,-0.077037,0.010305,0.010305,0.089742,0.002083,-0.890762,-0.000957,0.096335
52072,21716.0,1993-07-30,0.012120,0.0065,1.01,0.048386,138.698,64.101,3.5225,6.9,...,0.0024,0.0316,-0.077037,0.009720,0.009720,0.060382,-0.000387,0.433153,0.243196,0.066150
97720,31925.0,1993-07-30,0.006554,0.0064,1.15,0.003378,635.317,64.101,3.5225,6.9,...,0.0024,0.0316,-0.077037,0.004154,0.004154,0.083592,-0.003159,0.619454,0.019345,0.087479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102347,36888.0,2024-10-31,-0.011755,0.0205,0.38,-0.369789,1.400,120.731,2.5427,4.1,...,0.0047,0.0508,0.018446,-0.016455,-0.009595,-0.016542,-0.000514,-0.159942,-0.063420,-0.006659
176070,86513.0,2024-10-31,0.000691,0.0104,0.31,0.046618,549.600,120.731,2.5427,4.1,...,0.0047,0.0508,0.018446,-0.004009,-0.001237,-0.011957,-0.001242,-0.155974,0.013570,-0.002797
182998,95721.0,2024-10-31,-0.020548,0.0078,0.88,-0.080079,0.100,120.731,2.5427,4.1,...,0.0047,0.0508,0.018446,-0.025248,0.027648,-0.015228,0.000000,0.164118,-0.086403,-0.002461
182760,95714.0,2024-10-31,-0.020782,0.0103,0.88,-0.001780,4.500,120.731,2.5427,4.1,...,0.0047,0.0508,0.018446,-0.025482,0.027850,-0.015541,0.000000,0.164118,-0.001512,-0.002560


In [7]:
# Generates next date
def generate_next_date(list_of_dates, date):
    return list_of_dates[list_of_dates > date].min()

def process_factor_model(X_factor, y_factor, train_end, test_date):
    X_train = X_factor[X_factor['date_L1'] <= train_end].drop(columns='date_L1')
    X_test = X_factor[X_factor['date_L1'] == test_date].drop(columns='date_L1')

    y_train = y_factor.loc[X_train.index]
    y_test = y_factor.loc[X_test.index]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

#### **2. Model Training (Alpha)**

Asset-Pricing Factor Models:
* 5-Factor + MOM Model: mktrf, SMB, HML, RMW, CMA, UMD

The overall_function trains the models using the final selected parameters after hyperparameter tuning.

Sample code for Regime-Switching Model

Checking for Stationarity in the dataset

In [1]:
plt.figure(figsize=(12, 6))

for fundno, group in df.groupby('crsp_fundno'):
    plt.plot(group.index, group['rolling_alpha_5f'], label=f'Fund {fundno}')

plt.xlabel('Date')
plt.ylabel('Rolling Alpha (5-factor)')
plt.title('Rolling Alpha by Fund')
plt.legend(loc='best', fontsize='small', ncol=2)  # Adjust legend size for readability
plt.show()

NameError: name 'plt' is not defined

Overall Training Cycle Code

In [29]:
lstm_hyperparams_dict_healthcare = dict(
    {'2019': dict({
        'num_layers':3,
        'units':[96,32,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.2,0.1],
        'activation':['tanh','tanh','linear'],
        'lr':0.019962442034576384
    }),
     '2020': dict({
        'num_layers':3,
        'units':[64,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.1,0.1,0.1],
        'activation':['tanh','tanh','linear'],
        'lr':0.021504061608420576
    }),
     '2021': dict({
        'num_layers':3,
        'units':[96,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.1,0.1,0.1],
        'activation':['tanh','tanh','linear'],
        'lr':0.012302249555768368
    }),
     '2022': dict({
        'num_layers':3,
        'units':[96,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.1,0.2,0.2],
        'activation':['tanh','tanh','linear'],
        'lr':0.010092957472413086
    }),
     '2023': dict({
        'num_layers':2,
        'units':[64,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.2],
        'activation':['tanh','tanh'],
        'lr':0.018388080425636798
    }),
     '2024': dict({
        'num_layers':2,
        'units':[32,32],
        'optimizer':'Adam',
        'drop_out':[0.1,0.2],
        'activation':['tanh','tanh'],
        'lr':0.007947588699235507
    })}
)

rf_hyperparams_dict_healthcare = dict(
    {'2019': dict({
        'n_estimators':300,
        'min_samples_split':5,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':20
    }),
     '2020': dict({
        'n_estimators':100,
        'min_samples_split':10,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':None
    }),
     '2021': dict({
        'n_estimators':300,
        'min_samples_split':5,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':20
    }),
     '2022': dict({
        'n_estimators':100,
        'min_samples_split':10,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':None
    }),
     '2023': dict({
        'n_estimators':300,
        'min_samples_split':5,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':20
    }),
     '2024': dict({
        'n_estimators':100,
        'min_samples_split':10,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':None
    })}
)

Healthcare Mutual Funds

In [32]:
# Training Cycle
from tensorflow.keras.optimizers import Adam, Nadam, Adagrad

def overall_function(dataset, outcome, lstm_hyperparams_dict_healthcare, rf_hyperparams_dict_healthcare):
    dataset['date'] = pd.to_datetime(dataset['date']) # converting to date format
    dataset = dataset.sort_values(by='date')
    df_factor = dataset.drop(columns=['mkt_return','mth_return','rf']) # remove irrelevant variables

    # Creating Lagged and Stepped Datasets
    X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(df_factor, lag=1,target_var=outcome, id = 'crsp_fundno'),step=1,target_var=outcome, id = 'crsp_fundno_L1')
    # return(X_dataset)
    X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

    list_of_dates = pd.to_datetime(X_dataset['date_L1'])
    percentile_70 = list_of_dates.quantile(0.7) # 70-30 split
    train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
    df_end = list_of_dates.max()
    # return(y_dataset)
    results = []

    ## Implement cross-validation split
    tscv = TimeSeriesSplit(n_splits = 5)
    
    while train_end != df_end:
        
        test_date = generate_next_date(list_of_dates, train_end)
        if pd.isna(test_date):
            break 

        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
        
        # For Adding Results
        df_in_loop = y_test.copy()
        
        ### Model 1: Lasso Regression
        lasso_cv = LassoCV(cv = tscv, random_state = 18, max_iter = 100000)
        lasso_cv.fit(X_train, y_train)
        
        # Create the Lasso model with the optimal alpha value
        lasso_model = Lasso(alpha = lasso_cv.alpha_)
        lasso_model.fit(X_train, y_train)
        lassopred = lasso_model.predict(X_test)
        # Adding Linear Model
        df_in_loop[f'lasso'] = lassopred
        print('Lasso Done')
        
        ### Model 1: Ridge Regression ###
        ridge_cv = RidgeCV(cv = tscv)
        ridge_cv.fit(X_train, y_train)
    
        ridge_model = Ridge(alpha = ridge_cv.alpha_)
        ridge_model.fit(X_train, y_train)
        
        ridgepred = ridge_model.predict(X_test)
        # Adding Linear Model
        df_in_loop[f'ridge'] = ridgepred
        print('Ridge Done')
        
        ### Model 2: LSTM ###
        X_train_lstm = np.array(X_train).astype(np.float32)
        X_test_lstm = np.array(X_test).astype(np.float32)
        y_train_lstm = np.array(y_train).astype(np.float32)
        
        test_year = test_date.year
        print(f'test_year is {test_year} with hyperparameters of {lstm_hyperparams_dict_healthcare[str(test_year)]}')
        
        num_layers = len(lstm_hyperparams_dict_healthcare[str(test_year)]['units'])
        
        # Building LSTM Model
        lstm_model = Sequential()
        for layer_num in range(num_layers):
            is_last_layer = (layer_num == (num_layers - 1))  # Check if it's the last layer
            lstm_model.add(LSTM(
                units=lstm_hyperparams_dict_healthcare[str(test_year)]['units'][layer_num], 
                return_sequences=not is_last_layer,  # Only last layer has return_sequences=False
                input_shape=(X_train_lstm.shape[1], 1) if layer_num == 0 else None,  # Define input shape only for the first layer
                activation=lstm_hyperparams_dict_healthcare[str(test_year)]['activation'][layer_num]
            ))
            lstm_model.add(Dropout(lstm_hyperparams_dict_healthcare[str(test_year)]['drop_out'][layer_num]))
        # Output Layer
        lstm_model.add(Dense(units=1))
        
        # Compilation
        if lstm_hyperparams_dict_healthcare[str(test_year)]['optimizer'] == 'Adam':
            lstm_model.compile(
                optimizer=Adam(learning_rate=lstm_hyperparams_dict_healthcare[str(test_year)]['lr']),
                loss='mean_squared_error',
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError()
                ]
            )
        elif lstm_hyperparams_dict_healthcare[str(test_year)]['optimizer'] == 'Nadam':
            lstm_model.compile(
                optimizer=Nadam(learning_rate=lstm_hyperparams_dict_healthcare[str(test_year)]['lr']),
                loss='mean_squared_error',
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError()
                ]
            )

        # Early stopping callback
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

        # Train the model
        lstm_model.fit(
            X_train_lstm, y_train_lstm,
            epochs=10, batch_size=128,
            callbacks=[callback]
        )

        lstmpred = lstm_model.predict(X_test_lstm)
        df_in_loop[f'lstm'] = lstmpred
        print('LSTM Done')
        
        ### Model 3: Random Forest Regression ###
        print(f'test_year is {test_year} with hyperparameters of {rf_hyperparams_dict_healthcare[str(test_year)]}')
        rf_model = RandomForestRegressor(n_estimators=rf_hyperparams_dict_healthcare[str(test_year)]['n_estimators'], 
                                         min_samples_split=rf_hyperparams_dict_healthcare[str(test_year)]['min_samples_split'],
                                         min_samples_leaf=rf_hyperparams_dict_healthcare[str(test_year)]['min_samples_leaf'],
                                         max_features = rf_hyperparams_dict_healthcare[str(test_year)]['max_features'], 
                                         max_depth=rf_hyperparams_dict_healthcare[str(test_year)]['max_depth'],
                                         random_state=40, n_jobs=-1)

        rf_model.fit(X_train, y_train.values.ravel())
        rf_pred = rf_model.predict(X_test)
        # return(y_pred)
        df_in_loop[f'rf'] = rf_pred
        
        print('RF Done')
        
        ### Model 4: PCA ###
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)

        X_train_pure = X_train
        X_test_pure = X_test
        
        pca = PCA()
        pca.fit(X_train_pure)
        cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
          #to explain more than 85% of the variance
        num_components = np.where(cumulative_variance_ratio >= 0.85)[0][0] + 1 
        pca_new = PCA(n_components=num_components)
        X_train_pca = pca_new.fit_transform(X_train_pure)
        X_train_pca = pd.DataFrame(X_train_pca)
        X_train_pca.columns = X_train_pca.columns.astype(str)

        X_test_pca = pca_new.transform(X_test_pure)
        X_test_pca = pd.DataFrame(X_test_pca)
        X_test_pca.columns = X_test_pca.columns.astype(str)
        
        pure_factor_model = LinearRegression()
        pure_factor_model.fit(X_train_pca, y_train)
        y_pred = pure_factor_model.predict(X_test_pca)
        df_in_loop[f'pca'] = y_pred
        print('PCA Done')
        
        # return(y_pred)
        # Add results into loop
        results.append(df_in_loop)
        train_end = test_date
        num_remaining_dates = len(list(set(date for date in list_of_dates if date > test_date)))
        print(f'{num_remaining_dates} dates remaining')
    # return(results)
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df


In [33]:
# Run to train the models
y_new = overall_function(dataset=df, outcome = "rolling_alpha_5f", lstm_hyperparams_dict_healthcare=lstm_hyperparams_dict_healthcare, rf_hyperparams_dict_healthcare=rf_hyperparams_dict_healthcare)

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Lasso Done
Ridge Done
test_year is 2019 with hyperparameters of {'num_layers': 3, 'units': [96, 32, 32], 'optimizer': 'Adam', 'drop_out': [0.2, 0.2, 0.1], 'activation': ['tanh', 'tanh', 'linear'], 'lr': 0.019962442034576384}


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 63s 57ms/step - loss: 0.0026 - root_mean_squared_error: 0.0430
Epoch 2/10
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 54s 54ms/step - loss: 4.3946e-04 - root_mean_squared_error: 0.0210
Epoch 3/10
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 51s 51ms/step - loss: 4.3477e-04 - root_mean_squared_error: 0.0208
Epoch 4/10
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 52s 52ms/step - loss: 4.0944e-04 - root_mean_squared_error: 0.0202
Epoch 5/10
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - loss: 4.4204e-04 - root_mean_squared_error: 0.0210
Epoch 6/10
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - loss: 4.0997e-04 - root_mean_squared_error: 0.0202
Epoch 7/10
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 54s 54ms/step - loss: 4.3193e-04 - root_mean_squared_error: 0.0208
Epoch 8/10
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 54s 54ms/step - loss: 3.9463e-04 - root_mean_squared_error: 0.0199
Epoch 9/10
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 52s 52ms/step - loss: 4.3872e-04 - root_mean_squared_error: 0.0209
Epoch 10/10
1001/1001 ━

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Lasso Done
Ridge Done
test_year is 2021 with hyperparameters of {'num_layers': 3, 'units': [96, 32, 32], 'optimizer': 'Nadam', 'drop_out': [0.1, 0.1, 0.1], 'activation': ['tanh', 'tanh', 'linear'], 'lr': 0.012302249555768368}


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1110/1110 ━━━━━━━━━━━━━━━━━━━━ 61s 47ms/step - loss: 0.0011 - root_mean_squared_error: 0.0303
Epoch 2/10
1110/1110 ━━━━━━━━━━━━━━━━━━━━ 47s 43ms/step - loss: 5.2426e-04 - root_mean_squared_error: 0.0229
Epoch 3/10
1110/1110 ━━━━━━━━━━━━━━━━━━━━ 47s 42ms/step - loss: 4.7423e-04 - root_mean_squared_error: 0.0218
Epoch 4/10
1110/1110 ━━━━━━━━━━━━━━━━━━━━ 48s 43ms/step - loss: 4.6617e-04 - root_mean_squared_error: 0.0215
Epoch 5/10
1110/1110 ━━━━━━━━━━━━━━━━━━━━ 47s 42ms/step - loss: 4.3849e-04 - root_mean_squared_error: 0.0209
Epoch 6/10
 486/1110 ━━━━━━━━━━━━━━━━━━━━ 42s 68ms/step - loss: 4.2959e-04 - root_mean_squared_error: 0.0207

: 

In [48]:
# Run to train the models
y_new = overall_function(dataset=df, outcome = "rolling_alpha_5f")

Ridge Done


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 48s 140ms/step - loss: 6.2459e-04 - root_mean_squared_error: 0.0245
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 34s 136ms/step - loss: 4.3185e-04 - root_mean_squared_error: 0.0208
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 37s 147ms/step - loss: 4.4359e-04 - root_mean_squared_error: 0.0211
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 40s 159ms/step - loss: 4.3802e-04 - root_mean_squared_error: 0.0209
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 40s 158ms/step - loss: 3.9029e-04 - root_mean_squared_error: 0.0197
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 40s 161ms/step - loss: 3.9700e-04 - root_mean_squared_error: 0.0199
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 40s 157ms/step - loss: 4.0600e-04 - root_mean_squared_error: 0.0201
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 38s 153ms/step - loss: 3.9520e-04 - root_mean_squared_error: 0.0199
Epoch 9/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 40s 159ms/step - loss: 3.8697e-04 - root_mean_squared_error: 0.0197
Epoch 10/10
251/251 ━━━━━━━━

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 51s 156ms/step - loss: 6.9183e-04 - root_mean_squared_error: 0.0255
Epoch 2/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 39s 152ms/step - loss: 4.5251e-04 - root_mean_squared_error: 0.0213
Epoch 3/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 38s 151ms/step - loss: 4.2671e-04 - root_mean_squared_error: 0.0207
Epoch 4/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 38s 150ms/step - loss: 4.5154e-04 - root_mean_squared_error: 0.0212
Epoch 5/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 39s 154ms/step - loss: 4.1829e-04 - root_mean_squared_error: 0.0204
Epoch 6/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 43s 169ms/step - loss: 4.0652e-04 - root_mean_squared_error: 0.0202
Epoch 7/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 42s 165ms/step - loss: 4.2139e-04 - root_mean_squared_error: 0.0205
Epoch 8/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 43s 170ms/step - loss: 3.9514e-04 - root_mean_squared_error: 0.0199
Epoch 9/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 40s 156ms/step - loss: 3.9343e-04 - root_mean_squared_error: 0.0198
Epoch 10/10
253/253 ━━━━━━━━

...

Epoch 1/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 55s 134ms/step - loss: 6.4237e-04 - root_mean_squared_error: 0.0248
Epoch 2/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 49s 137ms/step - loss: 4.4120e-04 - root_mean_squared_error: 0.0210
Epoch 3/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 50s 141ms/step - loss: 4.7169e-04 - root_mean_squared_error: 0.0217
Epoch 4/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 4.2484e-04 - root_mean_squared_error: 0.0206
Epoch 5/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 3.8569e-04 - root_mean_squared_error: 0.0196
Epoch 6/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 49s 137ms/step - loss: 3.6205e-04 - root_mean_squared_error: 0.0190
Epoch 7/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 135ms/step - loss: 4.3256e-04 - root_mean_squared_error: 0.0208
Epoch 8/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 3.7134e-04 - root_mean_squared_error: 0.0193
Epoch 9/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 135ms/step - loss: 4.1982e-04 - root_mean_squared_error: 0.0205
Epoch 10/10
355/355 ━━━━━━━━

1/4 ━━━━━━━━━━━━━━━━━━━━ 2s 725ms/stepWARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001FA3975F400> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 261ms/step
LSTM Done
RF Done
PCA Done
1 dates remaining
Ridge Done


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 56s 136ms/step - loss: 7.0765e-04 - root_mean_squared_error: 0.0259
Epoch 2/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 4.5231e-04 - root_mean_squared_error: 0.0213
Epoch 3/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 4.4883e-04 - root_mean_squared_error: 0.0212
Epoch 4/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 4.2379e-04 - root_mean_squared_error: 0.0206
Epoch 5/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 135ms/step - loss: 4.2626e-04 - root_mean_squared_error: 0.0206
Epoch 6/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 4.1295e-04 - root_mean_squared_error: 0.0203
Epoch 7/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 4.2383e-04 - root_mean_squared_error: 0.0206
Epoch 8/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 4.2148e-04 - root_mean_squared_error: 0.0205
Epoch 9/10
355/355 ━━━━━━━━━━━━━━━━━━━━ 48s 136ms/step - loss: 3.7061e-04 - root_mean_squared_error: 0.0192
Epoch 10/10
355/355 ━━━━━━━━

In [ ]:
y_new.to_csv('tuned_results_lstm_rf_batchnum128.csv')

Technology Sector

In [9]:
from tensorflow.keras.optimizers import Adam, Adagrad

def overall_function(dataset, outcome):
    dataset['date'] = pd.to_datetime(dataset['date']) # converting to date format
    dataset = dataset.sort_values(by='date')
    df_factor = dataset.drop(columns=['mkt_return','mth_return','rf','rolling_sharpe', 'rolling_alpha_3f', 'rolling_alpha_4f']) # remove irrelevant variables

    # Creating Lagged and Stepped Datasets
    X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(df_factor, lag=1,target_var=outcome, id = 'crsp_fundno'),step=1,target_var=outcome, id = 'crsp_fundno_L1')
    # return(X_dataset)
    X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

    list_of_dates = pd.to_datetime(X_dataset['date_L1'])
    percentile_70 = list_of_dates.quantile(0.7) # 70-30 split
    train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
    df_end = list_of_dates.max()
    # return(y_dataset)
    results = []

    ## Implement cross-validation split
    tscv = TimeSeriesSplit(n_splits = 5)
    
    while train_end != df_end:
        
        test_date = generate_next_date(list_of_dates, train_end)
        if pd.isna(test_date):
            break 

        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
        
        # For Adding Results
        df_in_loop = y_test.copy()
        
        ### Model 1: Ridge Regression ###
        ridge_cv = RidgeCV(cv = tscv)
        ridge_cv.fit(X_train, y_train)
    
        ridge_model = Ridge(alpha = ridge_cv.alpha_)
        ridge_model.fit(X_train, y_train)
        
        ridgepred = ridge_model.predict(X_test)
        # Adding Linear Model
        df_in_loop[f'ridge'] = ridgepred
        print('Ridge Done')
        
        ### Model 2: LSTM ###
        X_train_lstm = np.array(X_train).astype(np.float32)
        X_test_lstm = np.array(X_test).astype(np.float32)
        y_train_lstm = np.array(y_train).astype(np.float32)
# 'num_layers': 4, 'units_1': 128, 'dropout_1': 0.1, 'lstm_units_1': 32, 'activation': 'tanh', 'learning_rate': 0.0064248322516769025, 'lstm_units_2': 32, 'dropout_2': 0.1, 'lstm_units_3': 32, 'dropout_3': 0.2
        
        # Define LSTM Model
        lstm_model = Sequential()
        # Layer 1
        lstm_model.add(LSTM(units = 128, return_sequences=True,input_shape=(X_train_lstm.shape[1], 1), activation='tanh'))
        lstm_model.add(Dropout(0.1))
        # Layer 2
        lstm_model.add(LSTM(units = 32, return_sequences=True, activation='tanh'))
        lstm_model.add(Dropout(0.1))
        # Layer 3 
        lstm_model.add(LSTM(units = 32, return_sequences=True, activation='tanh'))
        lstm_model.add(Dropout(0.2))
        # Layer 3 (Final Layer)
        lstm_model.add(LSTM(units = 32, return_sequences=False, activation='tanh'))
        lstm_model.add(Dropout(0.2))
        # Output Layer
        lstm_model.add(Dense(units=1))
        lstm_model.compile(
            optimizer=Adam(learning_rate=0.0064248322516769025),
            loss='mean_squared_error',
            metrics=[
                tf.keras.metrics.RootMeanSquaredError()
            ]
        )

        # Early stopping callback
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

        # Train the model
        lstm_model.fit(
            X_train_lstm, y_train_lstm,
            epochs=10, batch_size=512,
            callbacks=[callback]
        )

        lstmpred = lstm_model.predict(X_test_lstm)
        df_in_loop[f'lstm'] = lstmpred
        print('LSTM Done')
        
        ### Model 4: Random Forest Regression ###
        rf_model = RandomForestRegressor(n_estimators=100, 
                                         min_samples_split=5,
                                         min_samples_leaf=4,
                                         max_features = 'log2', 
                                         max_depth=30,
                                         random_state=40, n_jobs=-1)

        rf_model.fit(X_train, y_train.values.ravel())
        rf_pred = rf_model.predict(X_test)
        df_in_loop[f'rf'] = rf_pred
        print('RF Done')
        
        ### Model 5: PCA ###
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)

        X_train_pure = X_train
        X_test_pure = X_test
        
        pca = PCA()
        pca.fit(X_train_pure)
        cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
          #to explain more than 85% of the variance
        num_components = np.where(cumulative_variance_ratio >= 0.85)[0][0] + 1 
        pca_new = PCA(n_components=num_components)
        X_train_pca = pca_new.fit_transform(X_train_pure)
        X_train_pca = pd.DataFrame(X_train_pca)
        X_train_pca.columns = X_train_pca.columns.astype(str)

        X_test_pca = pca_new.transform(X_test_pure)
        X_test_pca = pd.DataFrame(X_test_pca)
        X_test_pca.columns = X_test_pca.columns.astype(str)
        
        pure_factor_model = LinearRegression()
        pure_factor_model.fit(X_train_pca, y_train)
        y_pred = pure_factor_model.predict(X_test_pca)
        df_in_loop[f'pca'] = y_pred
        print('PCA Done')
        
        # Add results into loop
        results.append(df_in_loop)
        train_end = test_date
        num_remaining_dates = len(list(set(date for date in list_of_dates if date > test_date)))
        print(f'{num_remaining_dates} dates remaining')
        
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df

In [59]:
# Run to train the models
y_tech = overall_function(dataset=df_tech, outcome = "rolling_alpha_5f")

Ridge Done


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 22s 166ms/step - loss: 0.0018 - root_mean_squared_error: 0.0401
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 139ms/step - loss: 5.3125e-04 - root_mean_squared_error: 0.0230
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 9s 114ms/step - loss: 5.2977e-04 - root_mean_squared_error: 0.0230
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 134ms/step - loss: 5.1906e-04 - root_mean_squared_error: 0.0228
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 143ms/step - loss: 5.4826e-04 - root_mean_squared_error: 0.0234
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 138ms/step - loss: 5.3902e-04 - root_mean_squared_error: 0.0232
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 9s 119ms/step - loss: 5.0143e-04 - root_mean_squared_error: 0.0224
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 150ms/step - loss: 4.9467e-04 - root_mean_squared_error: 0.0222
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 163ms/step - loss: 4.9041e-04 - root_mean_squared_error: 0.0221
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 163ms/ste

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 23s 163ms/step - loss: 0.0013 - root_mean_squared_error: 0.0353
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 162ms/step - loss: 5.3461e-04 - root_mean_squared_error: 0.0231
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 161ms/step - loss: 5.2565e-04 - root_mean_squared_error: 0.0229
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 163ms/step - loss: 5.2392e-04 - root_mean_squared_error: 0.0229
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 160ms/step - loss: 5.0632e-04 - root_mean_squared_error: 0.0225
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 140ms/step - loss: 4.9540e-04 - root_mean_squared_error: 0.0223
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 137ms/step - loss: 5.3353e-04 - root_mean_squared_error: 0.0231
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 160ms/step - loss: 4.9904e-04 - root_mean_squared_error: 0.0223
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 161ms/step - loss: 5.2226e-04 - root_mean_squared_error: 0.0228
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 162ms/s

...

Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 21s 125ms/step - loss: 0.0011 - root_mean_squared_error: 0.0318
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 126ms/step - loss: 5.2368e-04 - root_mean_squared_error: 0.0229
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 15s 136ms/step - loss: 5.0861e-04 - root_mean_squared_error: 0.0225
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 15s 141ms/step - loss: 4.9659e-04 - root_mean_squared_error: 0.0223
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 15s 142ms/step - loss: 4.5796e-04 - root_mean_squared_error: 0.0214
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 15s 144ms/step - loss: 4.6792e-04 - root_mean_squared_error: 0.0216
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 130ms/step - loss: 4.7671e-04 - root_mean_squared_error: 0.0218
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 15s 142ms/step - loss: 4.4272e-04 - root_mean_squared_error: 0.0210
Epoch 9/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 15s 138ms/step - loss: 4.5909e-04 - root_mean_squared_error: 0.0214
Epoch 10/10
107/107 ━━━━━━━━━━━━

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 23s 131ms/step - loss: 0.0013 - root_mean_squared_error: 0.0345
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 134ms/step - loss: 5.2686e-04 - root_mean_squared_error: 0.0229
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 135ms/step - loss: 5.0080e-04 - root_mean_squared_error: 0.0224
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 133ms/step - loss: 5.0291e-04 - root_mean_squared_error: 0.0224
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 129ms/step - loss: 4.9448e-04 - root_mean_squared_error: 0.0222
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 15s 143ms/step - loss: 4.8649e-04 - root_mean_squared_error: 0.0221
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 130ms/step - loss: 4.8455e-04 - root_mean_squared_error: 0.0220
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 130ms/step - loss: 4.9193e-04 - root_mean_squared_error: 0.0222
Epoch 9/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 127ms/step - loss: 4.5706e-04 - root_mean_squared_error: 0.0214
Epoch 10/10
107/107 ━━━━━━━━━━━━

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 22s 122ms/step - loss: 9.6428e-04 - root_mean_squared_error: 0.0301
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 128ms/step - loss: 4.9062e-04 - root_mean_squared_error: 0.0221
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 130ms/step - loss: 5.1397e-04 - root_mean_squared_error: 0.0227
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 125ms/step - loss: 4.8762e-04 - root_mean_squared_error: 0.0221
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 129ms/step - loss: 4.7845e-04 - root_mean_squared_error: 0.0219
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 132ms/step - loss: 4.5807e-04 - root_mean_squared_error: 0.0214
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 15s 141ms/step - loss: 4.8471e-04 - root_mean_squared_error: 0.0220
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step - loss: 4.6105e-04 - root_mean_squared_error: 0.0215
Epoch 9/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 129ms/step - loss: 4.4367e-04 - root_mean_squared_error: 0.0211
Epoch 10/10
107/107 ━━━━━━━━

In [60]:
y_tech.to_csv("tech_funds_forecast.csv")

Random Forest Hyperparameter Tuning (Healthcare)

In [10]:
from sklearn.model_selection import RandomizedSearchCV
tscv = TimeSeriesSplit(n_splits = 5)

df['date'] = pd.to_datetime(df['date']) # converting to date format
df = df.sort_values(by='date')
df_factor = df.drop(columns=['mkt_return','mth_return','rf']) # remove irrelevant variables

# Creating Lagged and Stepped Datasets
X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(df_factor, lag=1,target_var='rolling_alpha_5f', id = 'crsp_fundno'),step=1,target_var='rolling_alpha_5f', id = 'crsp_fundno_L1')
# return(X_dataset)
X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

list_of_dates = pd.to_datetime(X_dataset['date_L1'])
percentile_70 = list_of_dates.quantile(0.7) # 70-30 split

train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
df_end = list_of_dates.max()

# Define hyperparameter grid
rf_param_grid = {
    'n_estimators': [100, 300, 500, 1000],  # Number of trees
    'max_depth': [10, 20, 30, None],  # Depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples for split
    'min_samples_leaf': [1, 2, 4],  # Minimum samples per leaf
    'max_features': ['sqrt', 'log2', None]  # Number of features considered per split
}

def rf_tuner(X_train, y_train, rf_param_grid):
    # Initialize RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=40, n_jobs=-1)

    # Use RandomizedSearchCV for efficiency
    rf_random_search = RandomizedSearchCV(
        estimator=rf_model,
        param_distributions=rf_param_grid,
        n_iter=20,  # Number of parameter settings tested
        cv=tscv,  # TimeSeriesSplit
        scoring='neg_mean_squared_error',
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    # Perform hyperparameter tuning
    rf_random_search.fit(X_train, y_train.values.ravel())

    # Best hyperparameters
    best_rf_params = rf_random_search.best_params_
    print(f'Best RF Parameters: {best_rf_params}')
    return(best_rf_params)

In [ ]:
hyperparam_lst = []

# 2019
test_date = generate_next_date(list_of_dates, train_end)
X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
initial_best_rf_params = rf_tuner(X_train, y_train, rf_param_grid)
hyperparam_lst.append(initial_best_rf_params)
current_year = 2019

while pd.notna(test_date):
    test_date = generate_next_date(list_of_dates, train_end)
    if test_date.year != (current_year + 1): # Still same year: skip and repeat until the year is the next year
        train_end = test_date
        print(f"Train End is {train_end}, year is {train_end.year}, which is same as current year {current_year}. Repeating until following year is obtained")
        continue
    else:
        print(f"Test Date is {test_date}, year is {test_date.year}, which means it is being trained until the end of {current_year}. Conducting Tuning")
        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
        best_rf_params = rf_tuner(X_train, y_train, rf_param_grid)
        hyperparam_lst.append(best_rf_params)
        train_end = test_date # after tuning, expand the training set
        current_year = current_year + 1

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best RF Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 20}
Train End is 2019-12-31 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Test Date is 2020-01-31 00:00:00, year is 2020, which means it is being trained until the end of 2019. Conducting Tuning
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best RF Parameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': None}
Train End is 2020-02-28 00:00:00, year is 2020, which is same as current year 2020. Repeating until following year is obtained
Train End is 2020-03-31 00:00:00, year is 2020, which is same as current year 2020. Repeating until following year is obtained
Train End is 2020-04-30 00:00:00, year is 2020, which is same as current year 2020. Repeating until following y

LSTM Hyperparam Tuning Functions

In [11]:
from tensorflow.keras.optimizers import Adam, Adagrad, Nadam
def build_lstm_model(hp):
    lstm_model = Sequential()
    # Select the number of LSTM layers (2, 3, or 4)
    num_layers = hp.Choice('num_layers', [2, 3, 4])
    # Choose optimizer
    optimizer_name = hp.Choice('optimizer', ['Adam', 'Adagrad', 'Nadam'])
    optimizers = {
        "Adam": Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Adagrad": Adagrad(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Nadam": Nadam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG'))
    }
    # Layer 1
    first_layer_units = hp.Int('lstm_units_1', min_value=32, max_value=128, step=32)
    lstm_model.add(LSTM(
        units=first_layer_units,
        return_sequences=True,
        input_shape=(X_train_lstm.shape[1], 1),
        activation=hp.Choice(f'activation_1', ['tanh', 'sigmoid', 'linear'])
    ))
    lstm_model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.3, step=0.1)))
    
    # Add remaining layers with non-increasing units
    previous_units = first_layer_units  # Track previous layer units
    
    for i in range(1, num_layers):
        available_units = [u for u in [32, 64, 96, 128] if u <= previous_units]  # Only allow non-increasing sizes
        current_units = hp.Choice(f'lstm_units_{i+1}', available_units)

        lstm_model.add(LSTM(
            units=current_units,
            return_sequences=True if i < num_layers - 1 else False,  
            activation=hp.Choice(f'activation_{i+1}', ['tanh', 'sigmoid', 'linear'])
        ))
        lstm_model.add(Dropout(hp.Float(f'dropout_{i+1}', min_value=0.1, max_value=0.3, step=0.1)))
        previous_units = current_units  # Update tracking variable
    
    # Output Layer
    lstm_model.add(Dense(units=1))
    
    # Compile the model
    lstm_model.compile(
        optimizer=optimizers[optimizer_name],
        loss='mean_squared_error',
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    return lstm_model

def tune_lstm(X_train_lstm, y_train_lstm, year, save_path="best_lstm_model"):
    tuner = kt.BayesianOptimization(
        build_lstm_model,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=1,
        directory=f'lstm_tuning_{year}',
        project_name='fund_forecasting'
    )

    # Perform the search with batch_size as a hyperparameter to tune
    tuner.search(
        X_train_lstm, y_train_lstm,
        epochs=10,
        validation_split=0.2,
        batch_size=128,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
    )

    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    
    # Print the best hyperparameters
    print("Best Hyperparameters:", best_hps.values)

    # Build and train the best model using the best hyperparameters
    # best_model = tuner.hypermodel.build(best_hps)
    # best_model.fit(X_train_lstm, y_train_lstm, epochs=20, validation_split=0.2, batch_size=512,
    #                callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])
    
    # # Create directory if it doesn't exist
    # if not os.path.exists(save_path):
    #     os.makedirs(save_path)

    # # Save the best model
    # model_path = os.path.join(save_path, f"{lstm_model_name}.keras")
    # best_model.save(model_path)
    # print(f"Best model saved at {model_path}")

    return best_hps

LSTM Hyperparameter Tuning for 2019 - 2024 --> Yearly Hyperparameter Tuning

In [27]:
df['date'] = pd.to_datetime(df['date']) # converting to date format
df = df.sort_values(by='date')
df_factor = df.drop(columns=['mkt_return','mth_return','rf']) # remove irrelevant variables

# Creating Lagged and Stepped Datasets
X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(df_factor, lag=1,target_var='rolling_alpha_5f', id = 'crsp_fundno'),step=1,target_var='rolling_alpha_5f', id = 'crsp_fundno_L1')
# return(X_dataset)
X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

list_of_dates = pd.to_datetime(X_dataset['date_L1'])
percentile_70 = list_of_dates.quantile(0.7) # 70-30 split

train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
df_end = list_of_dates.max()

hyperparam_lst = []

# 2019
current_year = 2018
test_date = generate_next_date(list_of_dates, train_end)
X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
X_train_lstm = np.array(X_train).astype(np.float32)
X_test_lstm = np.array(X_test).astype(np.float32)
y_train_lstm = np.array(y_train).astype(np.float32)
# best_model_params = tune_lstm(X_train_lstm, y_train_lstm, year = current_year)
# hyperparam_lst.append(best_model_params)


while pd.notna(test_date):
    test_date = generate_next_date(list_of_dates, train_end)
    if test_date.year != (current_year + 1): # Still same year: skip and repeat until the year is the next year
        train_end = test_date
        print(f"Train End is {train_end}, year is {train_end.year}, which is same as current year {current_year}. Repeating until following year is obtained")
        continue
    else:
        print(f"Test Date is {test_date}, year is {test_date.year}, which means it is being trained until the end of {current_year}. Conducting Tuning")
        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
        X_train_lstm = np.array(X_train).astype(np.float32)
        X_test_lstm = np.array(X_test).astype(np.float32)
        y_train_lstm = np.array(y_train).astype(np.float32)
        best_model_params = tune_lstm(X_train_lstm, y_train_lstm, year = current_year+1)
        hyperparam_lst.append(best_model_params)
        train_end = test_date # after tuning, expand the training set
        current_year = current_year + 1
# df_end.year

Test Date is 2019-12-31 00:00:00, year is 2019, which means it is being trained until the end of 2018. Conducting Tuning
Reloading Tuner from lstm_tuning_2019\fund_forecasting\tuner0.json
Best Hyperparameters: {'num_layers': 3, 'optimizer': 'Adam', 'learning_rate': 0.019962442034576384, 'lstm_units_1': 96, 'dropout_1': 0.2, 'lstm_units_2': 32, 'activation': 'linear', 'dropout_2': 0.2, 'lstm_units_3': 32, 'dropout_3': 0.1, 'lstm_units_4': 32, 'dropout_4': 0.1}
Test Date is 2020-01-31 00:00:00, year is 2020, which means it is being trained until the end of 2019. Conducting Tuning
Reloading Tuner from lstm_tuning_2020\fund_forecasting\tuner0.json
Best Hyperparameters: {'num_layers': 3, 'optimizer': 'Nadam', 'learning_rate': 0.021504061608420576, 'lstm_units_1': 64, 'dropout_1': 0.1, 'lstm_units_2': 32, 'activation': 'linear', 'dropout_2': 0.1, 'lstm_units_3': 32, 'dropout_3': 0.1, 'lstm_units_4': 32, 'dropout_4': 0.2}
Train End is 2020-02-28 00:00:00, year is 2020, which is same as curren

In [26]:
print(hyperparam_lst[3].values)

{'num_layers': 2, 'optimizer': 'Adam', 'learning_rate': 0.018388080425636798, 'lstm_units_1': 64, 'dropout_1': 0.2, 'lstm_units_2': 32, 'activation': 'tanh', 'dropout_2': 0.2, 'lstm_units_3': 32, 'dropout_3': 0.2, 'lstm_units_4': 32, 'dropout_4': 0.2, 'activation_1': 'tanh', 'activation_2': 'tanh', 'activation_3': 'linear', 'activation_4': 'sigmoid', 'activation_final': 'l'}


In [44]:
# Load the best model
best_model = tf.keras.models.load_model('best_lstm_model2/best_lstm_model.keras')

# Make predictions
predictions = best_model.predict(X_test_lstm)
predictions

34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step


array([[-0.0009765 ],
       [-0.00477923],
       [-0.00147857],
       ...,
       [-0.00080813],
       [-0.00748954],
       [ 0.00452702]], dtype=float32)